## Load in patients dataset

In [1]:
import dask.dataframe as dd
import os

## Path to the directory where the data is located
data_dir = r'R:\Kevin\Data\MIMIC'

In [2]:
# Load the patients.csv file using Dask
patients_df = dd.read_csv(os.path.join(data_dir, 'patients.csv'))

# Show some basic information
print("Dask dataframe created for patients.csv")
print("Number of partitions:", patients_df.npartitions)
print(patients_df.head())

Dask dataframe created for patients.csv
Number of partitions: 1
   subject_id gender  anchor_age  anchor_year anchor_year_group         dod
0    10000032      F          52         2180       2014 - 2016  2180-09-09
1    10000048      F          23         2126       2008 - 2010         NaN
2    10000058      F          33         2168       2020 - 2022         NaN
3    10000068      F          19         2160       2008 - 2010         NaN
4    10000084      M          72         2160       2017 - 2019  2161-02-13


## Load in lab events dataset

In [27]:
# Load the labevents.csv file using Dask
labevents_df = dd.read_csv(os.path.join(data_dir, 'labevents.csv'))

# Show some basic information
print("Dask dataframe created for labevents.csv")
print("Number of partitions:", labevents_df.npartitions)
print(labevents_df.head())

Dask dataframe created for labevents.csv
Number of partitions: 287
   labevent_id  subject_id  hadm_id  specimen_id  itemid order_provider_id  \
0            1    10000032      NaN      2704548   50931            P69FQC   
1            2    10000032      NaN     36092842   51071            P69FQC   
2            3    10000032      NaN     36092842   51074            P69FQC   
3            4    10000032      NaN     36092842   51075            P69FQC   
4            5    10000032      NaN     36092842   51079            P69FQC   

             charttime            storetime value  valuenum valueuom  \
0  2180-03-23 11:51:00  2180-03-23 15:56:00   ___      95.0    mg/dL   
1  2180-03-23 11:51:00  2180-03-23 16:00:00   NEG       NaN      NaN   
2  2180-03-23 11:51:00  2180-03-23 16:00:00   NEG       NaN      NaN   
3  2180-03-23 11:51:00  2180-03-23 16:00:00   NEG       NaN      NaN   
4  2180-03-23 11:51:00  2180-03-23 16:00:00   NEG       NaN      NaN   

   ref_range_lower  ref_range_u

## Load in lab items dataset

In [22]:
# Load the d_labitems.csv file using Dask
d_labitems_df = dd.read_csv(os.path.join(data_dir, 'd_labitems.csv'))

# Show some basic information
print("Dask dataframe created for d_labitems.csv")
print("Number of partitions:", d_labitems_df.npartitions)
print(d_labitems_df.head())

Dask dataframe created for d_labitems.csv
Number of partitions: 1
   itemid                                label  fluid   category
0   50801           Alveolar-arterial Gradient  Blood  Blood Gas
1   50802                          Base Excess  Blood  Blood Gas
2   50803  Calculated Bicarbonate, Whole Blood  Blood  Blood Gas
3   50804                 Calculated Total CO2  Blood  Blood Gas
4   50805                    Carboxyhemoglobin  Blood  Blood Gas


## Load in admissions df

In [6]:
# Load admissions.csv.gz
admissions_df = dd.read_csv(os.path.join(data_dir, 'admissions.csv'))
print("Dask dataframe created for admissions.csv.gz")
print("Number of partitions:", admissions_df.npartitions)


Dask dataframe created for admissions.csv.gz
Number of partitions: 1


In [7]:
admissions_df

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
npartitions=1,,,,,,,,,,,,,,,,
,int64,int64,string,string,float64,string,string,string,string,string,string,string,string,string,string,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## Determined tests of interest

In [8]:
# Updated test lists according to routine tests in outpatient settings
cbc_tests = ['WBC', 'WHITE BLOOD CELL', 'RED BLOOD CELL', 'RBC', 'HEMOGLOBIN', 'HEMATOCRIT', 'PLATELET', 'MCV', 'MCH', 'MCHC', 
             'RDW', 'NEUTROPHIL', 'LYMPHOCYTES', 'MONOCYTES', 'EOSINOPHILS', 'BASOPHILS', 'GRANULOCYTES']
cmp_tests = ['SODIUM', 'POTASSIUM', 'CHLORIDE', 'BICARBONATE', 'BUN', 'CREATININE', 'GLUCOSE', 
             'CALCIUM', 'TOTAL PROTEIN', 'ALBUMIN', 'GLOBULIN', 'BILIRUBIN', 'ALKALINE PHOSPHATASE', 'ALT', 'AST']
lipids_tests = ['CHOLESTEROL', 'LDL', 'HDL', 'TRIGLYCERIDES']
diabetes_tests = ['HEMOGLOBIN A1C', 'HbAIC']
thyroid_tests = ['TSH', 'FREE T4', 'THYROXINE', 'TRIIODOTHYRONINE']
urine_tests = ['URINALYSIS', 'MICROALBUMIN', 'CREATININE RATIO', 'YEAST','HYALINE', 'PROTEIN', 'GLUCOSE', 'KETONE',
               'UROBILINOGEN','MCNC','NITRITE','LEUKOCYTE ESTERASE']
surgery_tests = ['PT', 'INR']

# Combine all test names
all_tests = cbc_tests + cmp_tests + lipids_tests + diabetes_tests + thyroid_tests + urine_tests + surgery_tests

# Create a regex pattern to match any of the test labels as substrings (case-insensitive)
pattern = '|'.join([test.upper() for test in all_tests])

# Ensure no NaN values in the relevant columns (fill them with an empty string)
d_labitems_df = d_labitems_df.fillna({'label': '', 'fluid': '', 'category': ''})

# Filter d_labitems for labels that contain any of the routine test names
# and ensure the fluid is 'Blood' or 'Urine', and the category is either 'Chemistry' or 'Hematology'
filtered_tests = d_labitems_df[
    d_labitems_df['label'].str.upper().str.contains(pattern, regex=True) &
    (d_labitems_df['fluid'].str.upper().isin(['BLOOD', 'URINE'])) &
    (d_labitems_df['category'].str.upper().isin(['CHEMISTRY', 'HEMATOLOGY']))
]

# Compute the results
filtered_tests_result = filtered_tests.compute()

# Display the filtered tests
print(filtered_tests_result)



      itemid                           label  fluid   category
50     50852                % Hemoglobin A1c  Blood  Chemistry
53     50855             Absolute Hemoglobin  Blood  Chemistry
59     50861  Alanine Aminotransferase (ALT)  Blood  Chemistry
60     50862                         Albumin  Blood  Chemistry
61     50863            Alkaline Phosphatase  Blood  Chemistry
...      ...                             ...    ...        ...
1621   53162                          RDW-SD  Blood  Chemistry
1641   53182                            MCHC  Blood  Chemistry
1643   53184                             MCH  Blood  Chemistry
1647   53188                     Lymphocytes  Blood  Chemistry
1648   53189                  Platelet Count  Blood  Chemistry

[214 rows x 4 columns]


## Excluded certain tests based on John's input

In [9]:
# Exclude tests that we decide should not be included
excluded_itemids = [50889, 51100, 50950, 50935, 50949, 50951, 51097, 51078, 50864, 51067, 50881, 51005, 50982, 50877, 51224, 51077, 
                    50991, 51076, 51068, 51225, 51223, 51066, 51271, 51273, 51106, 51212, 51272, 51270, 51073, 52129, 51280, 51281]
filtered_tests_result = filtered_tests_result[~filtered_tests_result['itemid'].isin(excluded_itemids)]
# Display the filtered tests
print(filtered_tests_result)

      itemid                           label  fluid   category
50     50852                % Hemoglobin A1c  Blood  Chemistry
53     50855             Absolute Hemoglobin  Blood  Chemistry
59     50861  Alanine Aminotransferase (ALT)  Blood  Chemistry
60     50862                         Albumin  Blood  Chemistry
61     50863            Alkaline Phosphatase  Blood  Chemistry
...      ...                             ...    ...        ...
1621   53162                          RDW-SD  Blood  Chemistry
1641   53182                            MCHC  Blood  Chemistry
1643   53184                             MCH  Blood  Chemistry
1647   53188                     Lymphocytes  Blood  Chemistry
1648   53189                  Platelet Count  Blood  Chemistry

[182 rows x 4 columns]


## Crate lookup for patients table and a race lookup for each subject

In [23]:
# Assuming filtered_tests_result and other necessary Dask DataFrames (labevents_df, patients_df, admissions_df) are already loaded

# Convert d_labitems_df to pandas to map itemid to test names
lab_test_map = dict(zip(filtered_tests_result['itemid'], filtered_tests_result['label']))

# Convert patients_df to a lookup dictionary for demographics (using only the available columns)
patients_lookup = patients_df.set_index('subject_id')[['anchor_age', 'anchor_year', 'gender']].compute().to_dict(orient='index')

# Compute the admissions DataFrame to Pandas for easier manipulation
admissions_df_pandas = admissions_df[['subject_id', 'race']].compute()

# Group by subject_id and check for patients with multiple unique races
race_counts = admissions_df_pandas.groupby('subject_id')['race'].nunique()

# Create a race lookup dictionary:
# - If a patient has a single race, use that race.
# - If a patient has multiple races, set race to 'Unknown'.
race_lookup = {}
for subject_id, unique_races in race_counts.items():
    if unique_races == 1:
        race_lookup[subject_id] = admissions_df_pandas[admissions_df_pandas['subject_id'] == subject_id]['race'].iloc[0]
    else:
        race_lookup[subject_id] = 'Unknown'

## Merge patient data with lab events

In [38]:
from tqdm import tqdm
import dask.dataframe as dd
import pandas as pd
import pickle

labevents_df = labevents_df.merge(
    patients_df[['subject_id', 'anchor_age', 'anchor_year', 'gender']],
    on='subject_id', how='left'
)


# Function to process each partition of the labevents_df
def process_partition(partition):
    # Filter out rows that don't match our tests of interest and drop NaN values in valuenum
    partition = partition[partition['itemid'].isin(lab_test_map.keys()) & partition['valuenum'].notna()]
    
    # Add test names and compute the year of the test from charttime
    partition['test_name'] = partition['itemid'].map(lab_test_map)
    partition['test_year'] = partition['charttime'].str[:4].astype(int)
    
    # Ensure no null values in test_name and valueuom columns
    partition['test_name'] = partition['test_name'].fillna('')
    partition['valueuom'] = partition['valueuom'].fillna('')
    
    # Add the unit of measurement to the test name
    partition['test_name_with_unit'] = partition['test_name'] + ' (' + partition['valueuom'] + ')'
    
    # Compute the age at the time of the test using vectorized operations
    partition['age_at_test'] = partition['anchor_age'] + (partition['test_year'] - partition['anchor_year'])
    
    # Group by subject_id and aggregate results
    partition_data = {}
    
    for person_id, group in partition.groupby('subject_id'):
        demographics = [int(group.iloc[0]['anchor_age']), group.iloc[0]['gender'], race_lookup.get(person_id, 'Unknown')]
        
        # Create a dictionary for each test with (valuenum, age_at_test) tuples
        tests = group[['test_name_with_unit', 'valuenum', 'age_at_test']].groupby('test_name_with_unit').apply(
            lambda x: list(zip(x['valuenum'], x['age_at_test']))
        ).to_dict()
        
        # Add demographics to the test dictionary
        tests['demographics'] = demographics
        partition_data[person_id] = tests
    
    return partition_data

# Define the meta for Dask (it tells Dask the structure of the DataFrame)
meta = {
    'itemid': 'int64',
    'valuenum': 'float64',
    'valueuom': 'object',
    'test_name_with_unit': 'object',
    'test_year': 'int64',
    'anchor_age': 'int64',
    'anchor_year': 'int64',
    'gender': 'object',
    'charttime': 'object',
    'subject_id': 'int64'
}

# Initialize an empty dictionary to store the final results
patient_data = {}

# Get the number of partitions in the labevents_df
total_partitions = labevents_df.npartitions

# Initialize the progress bar
progress_bar = tqdm(total=total_partitions, desc="Processing Lab Event Partitions")

# Process each partition of the labevents_df and track progress
for partition_idx, partition_data in enumerate(labevents_df.map_partitions(process_partition, meta=meta).compute()):
    for person_id, data in partition_data.items():
        if person_id not in patient_data:
            patient_data[person_id] = data
        else:
            for test_name, measurements in data.items():
                if test_name == "demographics":
                    continue
                if test_name in patient_data[person_id]:
                    patient_data[person_id][test_name].extend(measurements)
                else:
                    patient_data[person_id][test_name] = measurements
    
    # Update progress bar
    progress_bar.update(1)

# Close the progress bar when done
progress_bar.close()

# Output the first 5 entries for verification
print(list(patient_data.items())[:5])

# Save patient_data as a pickle file for faster access later
with open('patient_data.pkl', 'wb') as f:
    pickle.dump(patient_data, f)

print("Patient data saved as patient_data.pkl")




   labevent_id  subject_id  hadm_id  specimen_id  itemid order_provider_id  \
0            1    10000032      NaN      2704548   50931            P69FQC   
1            2    10000032      NaN     36092842   51071            P69FQC   
2            3    10000032      NaN     36092842   51074            P69FQC   
3            4    10000032      NaN     36092842   51075            P69FQC   
4            5    10000032      NaN     36092842   51079            P69FQC   

             charttime            storetime value  valuenum  ...  \
0  2180-03-23 11:51:00  2180-03-23 15:56:00   ___      95.0  ...   
1  2180-03-23 11:51:00  2180-03-23 16:00:00   NEG       NaN  ...   
2  2180-03-23 11:51:00  2180-03-23 16:00:00   NEG       NaN  ...   
3  2180-03-23 11:51:00  2180-03-23 16:00:00   NEG       NaN  ...   
4  2180-03-23 11:51:00  2180-03-23 16:00:00   NEG       NaN  ...   

                                            comments  anchor_age_x  \
0  IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI...

Processing Lab Event Partitions: 100%|██████████| 287/287 [44:42<00:00,  9.35s/it]


[(10000032, {'Alanine Aminotransferase (ALT) (IU/L)': [(102.0, 52), (100.0, 52), (88.0, 52), (110.0, 52), (113.0, 52), (126.0, 52), (117.0, 52), (153.0, 52), (114.0, 52), (96.0, 52), (135.0, 52), (132.0, 52), (112.0, 52), (120.0, 52)], 'Albumin (g/dL)': [(3.3, 52), (3.3, 52), (3.3, 52), (2.8, 52), (2.7, 52), (3.6, 52), (3.8, 52), (3.5, 52), (3.5, 52), (3.5, 52), (3.3, 52)], 'Alkaline Phosphatase (IU/L)': [(109.0, 52), (114.0, 52), (103.0, 52), (116.0, 52), (117.0, 52), (117.0, 52), (110.0, 52), (114.0, 52), (86.0, 52), (69.0, 52), (123.0, 52), (112.0, 52), (101.0, 52), (115.0, 52)], 'Asparate Aminotransferase (AST) (IU/L)': [(143.0, 52), (114.0, 52), (98.0, 52), (162.0, 52), (182.0, 52), (158.0, 52), (275.0, 52), (187.0, 52), (168.0, 52), (244.0, 52), (221.0, 52), (176.0, 52), (206.0, 52)], 'Basophils (%)': [(0.5, 52), (0.8, 52), (0.5, 52), (0.5, 52), (0.3, 52), (0.5, 52), (0.2, 52)], 'Bicarbonate (mEq/L)': [(27.0, 52), (27.0, 52), (28.0, 52), (29.0, 52), (28.0, 52), (26.0, 52), (26.0,

## Code used to generate excel sheets with test counts 

In [ ]:


# Extract test itemids from filtered_tests_result to match in the sampled_labevents_df
test_itemids = filtered_tests_result['itemid'].unique()

# Create a mapping of itemid to test label (name), fluid, and category
itemid_to_label = dict(zip(filtered_tests_result['itemid'], filtered_tests_result['label']))
itemid_to_fluid = dict(zip(filtered_tests_result['itemid'], filtered_tests_result['fluid']))
itemid_to_category = dict(zip(filtered_tests_result['itemid'], filtered_tests_result['category']))

# Filter the sampled_labevents_df to include only rows with itemids that match the tests of interest
matched_tests_df = labevents_df[labevents_df['itemid'].isin(test_itemids)]

# Count the number of occurrences of each test (by itemid) in the sampled_labevents_df
test_counts = matched_tests_df.groupby(['itemid', 'valueuom']).size().reset_index(name='Count')

# Add test names, fluid, and category to the DataFrame using the itemid_to_label, itemid_to_fluid, and itemid_to_category mappings
test_counts['Test Name'] = test_counts['itemid'].map(itemid_to_label)
test_counts['Fluid'] = test_counts['itemid'].map(itemid_to_fluid)
test_counts['Category'] = test_counts['itemid'].map(itemid_to_category)

# Reorder the columns to display itemid, test name, fluid, category, unit of measure, and count
test_counts_df = test_counts[['itemid', 'Test Name', 'Fluid', 'Category', 'valueuom', 'Count']]

# Rename the columns
test_counts_df.columns = ['ItemID', 'Test Name', 'Fluid', 'Category', 'Unit of Measure', 'Count']

# Sort the DataFrame by the 'Count' column in decreasing order
test_counts_df_sorted = test_counts_df.sort_values(by='Count', ascending=False)

# Display the sorted DataFrame
print(test_counts_df_sorted)

# Export the sorted DataFrame to a CSv file and save it to the local directory
output_csv_path = os.path.join(data_dir, 'sorted_test_counts_with_category.csv')
test_counts_df_sorted.to_csv(output_csv_path, index=False)

print(f"CSV file saved at: {output_csv_path}")




     ItemID                   Test Name  Fluid    Category Unit of Measure  \
62    51221                  Hematocrit  Blood  Hematology               %   
21    50912                  Creatinine  Blood   Chemistry           mg/dL   
77    51265              Platelet Count  Blood  Hematology            K/uL   
63    51222                  Hemoglobin  Blood  Hematology            g/dL   
89    51301           White Blood Cells  Blood  Hematology            K/uL   
..      ...                         ...    ...         ...             ...   
98    51472        Cholesterol Crystals  Urine  Hematology            /hpf   
94    51468  Calcium Carbonate Crystals  Urine  Hematology            /hpf   
87    51281      Reptilase Time Control  Blood  Hematology             sec   
133   52410                 Fatty Casts  Urine  Hematology           #/lpf   
92    51465          Bilirubin Crystals  Urine  Hematology            /hpf   

       Count  
62   4331614  
21   4319090  
77   4214047  
63 